In [17]:
## VLS token for pushing
#

In [18]:
%pip install duckdb


Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [19]:
import os

path = "census_master.db"
print("cwd:", os.getcwd())
print("exists:", os.path.exists(path))
print("abs path:", os.path.abspath(path))
print("size bytes:", os.path.getsize(path))


cwd: /home/vascott/teams/z5/Vanessa/DSC-288R-CAPSTONE
exists: True
abs path: /home/vascott/teams/z5/Vanessa/DSC-288R-CAPSTONE/census_master.db
size bytes: 337129472


In [20]:
with open("census_master.db", "rb") as f:
    header = f.read(100)
print(header[:32])

b'g\xfd\x93(\x81\x8d.\xb3DUCK@\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00'


In [21]:
import duckdb
con = duckdb.connect("census_master.db", read_only=True)
con.execute("SHOW TABLES").fetchdf()


,name
0,census_masterdb_all
1,master_person_data


In [22]:
df_preview = con.execute("""
    SELECT *
    FROM census_masterdb_all
    LIMIT 10
""").fetchdf()

df_preview


,ADJINC,AGEP,ANC,ANC1P,ANC2P,CIT,CITWP,COW,DDRS,DEAR,...,SSIP,SSP,VPS,WAGP,WAOB,WKHP,WKL,WRK,YOEP,year
0,1013097,30,1,210,999,1,<NA>,6,2,2,...,0,0,None,500,1,40,1,<NA>,<NA>,2018
1,1013097,18,1,290,999,1,<NA>,<NA>,2,2,...,0,0,None,0,1,<NA>,3,2,<NA>,2018
2,1013097,69,2,902,917,1,<NA>,<NA>,2,2,...,0,0,06,0,1,<NA>,3,2,<NA>,2018
3,1013097,25,1,290,999,1,<NA>,<NA>,1,1,...,0,0,None,0,1,<NA>,3,2,<NA>,2018
4,1013097,31,1,924,999,1,<NA>,<NA>,2,2,...,0,0,None,0,1,<NA>,3,<NA>,<NA>,2018
5,1013097,19,1,706,999,5,<NA>,1,2,2,...,0,0,None,50,4,5,1,<NA>,2018,2018
6,1013097,21,2,148,050,1,<NA>,4,2,2,...,0,0,None,7700,1,20,1,1,<NA>,2018
7,1013097,65,1,050,999,1,<NA>,2,2,2,...,0,11300,None,5000,1,8,1,1,<NA>,2018
8,1013097,55,1,740,999,1,<NA>,<NA>,1,2,...,0,7200,None,0,1,<NA>,3,2,<NA>,2018
9,1013097,82,4,999,999,4,1940,<NA>,1,2,...,0,0,None,0,4,<NA>,3,<NA>,2005,2018


In [23]:

import pandas as pd
con = duckdb.connect("census_master.db", read_only=True)

TABLE = "census_masterdb_all"
TARGET = "POVPIP"  # make sure casing matches your table

# 1) Get numeric columns (DuckDB types)
schema = con.execute(f"DESCRIBE {TABLE}").fetchdf()

numeric_types = {
    "TINYINT", "SMALLINT", "INTEGER", "BIGINT",
    "UTINYINT", "USMALLINT", "UINTEGER", "UBIGINT",
    "HUGEINT", "UHUGEINT",
    "FLOAT", "REAL", "DOUBLE",
    "DECIMAL"
}

num_cols = (
    schema[schema["column_type"].str.upper().str.split("(").str[0].isin(numeric_types)]
    ["column_name"]
    .tolist()
)

# drop the target itself if present
num_cols = [c for c in num_cols if c.upper() != TARGET.upper()]

print(f"Found {len(num_cols)} numeric columns (excluding {TARGET}).")

# 2) Compute correlations in SQL (pairwise complete cases)
rows = []
for c in num_cols:
    # corr() ignores rows where either side is NULL, but will be NULL if not enough variance/data
    q = f"""
    SELECT
        '{c}' AS variable,
        corr("{c}", "{TARGET}") AS corr_value
    FROM {TABLE}
    WHERE "{c}" IS NOT NULL AND "{TARGET}" IS NOT NULL
    """
    corr_val = con.execute(q).fetchone()[1]
    rows.append((c, corr_val))

corr_df = pd.DataFrame(rows, columns=["variable", "corr_value"])

# 3) Rank by absolute correlation, drop nulls, show top 50
top50 = (
    corr_df.dropna(subset=["corr_value"])
           .assign(abs_corr=lambda d: d["corr_value"].abs())
           .sort_values("abs_corr", ascending=False)
           .head(50)
           .reset_index(drop=True)
)

top50


Found 88 numeric columns (excluding POVPIP).


,variable,corr_value,abs_corr
0,PRIVCOV,-0.470463,0.470463
1,HINS4,0.454435,0.454435
2,HINS1,-0.438577,0.438577
3,PINCP,0.416190,0.416190
4,PERNP,0.371724,0.371724
5,WAGP,0.362770,0.362770
6,PUBCOV,0.349636,0.349636
7,WRK,-0.265203,0.265203
8,ESR,-0.249660,0.249660
9,WKL,-0.244975,0.244975


In [24]:

con = duckdb.connect("census_master.db", read_only=True)

TABLE = "census_masterdb_all"
TARGET = "POVPIP"

schema = con.execute(f"DESCRIBE {TABLE}").fetchdf()

# treat VARCHAR as categorical
cat_cols = schema[
    schema["column_type"].str.upper().str.startswith("VARCHAR")
]["column_name"].tolist()

# remove obvious IDs / free text if needed
exclude = {TARGET, "SERIALNO", "PUMA", "YEAR"}
cat_cols = [c for c in cat_cols if c not in exclude]

print(f"Found {len(cat_cols)} categorical variables.")



Found 30 categorical variables.


In [25]:
schema = con.execute("DESCRIBE census_masterdb_all").fetchdf()
schema.head()



,column_name,column_type,null,key,default,extra
0,ADJINC,BIGINT,YES,None,None,None
1,AGEP,BIGINT,YES,None,None,None
2,ANC,BIGINT,YES,None,None,None
3,ANC1P,VARCHAR,YES,None,None,None
4,ANC2P,VARCHAR,YES,None,None,None


In [26]:
# 1) What type is POVPIP?
schema[schema["column_name"].str.upper().eq("POVPIP")]


,column_name,column_type,null,key,default,extra
77,POVPIP,BIGINT,YES,None,None,None


In [27]:
# 2) How many VARCHAR columns exist?
schema["column_type"].value_counts().head(15)


column_type
BIGINT     89
VARCHAR    32
Name: count, dtype: int64

In [28]:
# 3) Is POVPIP mostly null?
con.execute("""
SELECT
  COUNT(*) AS n_rows,
  SUM(CASE WHEN POVPIP IS NULL THEN 1 ELSE 0 END) AS n_null_povpip
FROM census_masterdb_all
""").fetchdf()


,n_rows,n_null_povpip
0,1928458,70832.0


In [29]:
import pandas as pd

TABLE = "census_masterdb_all"
TARGET = "POVPIP"
MAX_LEVELS = 30      # tweak: 20–50 is common
MIN_NON_NULL = 5000  # avoid tiny/sparse columns

schema = con.execute(f"DESCRIBE {TABLE}").fetchdf()
all_cols = schema["column_name"].tolist()
all_cols = [c for c in all_cols if c.upper() != TARGET.upper()]

candidates = []
for c in all_cols:
    q = f"""
    SELECT
      COUNT("{c}") AS non_null,
      COUNT(DISTINCT "{c}") AS n_levels
    FROM {TABLE}
    WHERE "{c}" IS NOT NULL
    """
    non_null, n_levels = con.execute(q).fetchone()
    if non_null >= MIN_NON_NULL and 2 <= n_levels <= MAX_LEVELS:
        candidates.append(c)

print("Candidate categorical-ish columns:", len(candidates))
candidates[:20]


Candidate categorical-ish columns: 83


['ADJINC',
 'ANC',
 'CIT',
 'COW',
 'DDRS',
 'DEAR',
 'DECADE',
 'DEYE',
 'DIS',
 'DOUT',
 'DPHY',
 'DRAT',
 'DRATX',
 'DREM',
 'DRIVESP',
 'ENG',
 'ESP',
 'ESR',
 'GCL',
 'GCM']

In [30]:
# Look at categorical VARCHAR variables and there aggregated correlation to POVPIP
def eta_squared_any(con, table, target, cat):
    q = f"""
    WITH base AS (
      SELECT
        TRY_CAST("{target}" AS DOUBLE) AS y,
        CAST("{cat}" AS VARCHAR) AS g
      FROM {table}
      WHERE "{target}" IS NOT NULL AND "{cat}" IS NOT NULL
    ),
    overall AS (
      SELECT AVG(y) AS grand_mean
      FROM base
      WHERE y IS NOT NULL
    ),
    by_group AS (
      SELECT g, COUNT(*) AS n, AVG(y) AS group_mean
      FROM base
      WHERE y IS NOT NULL
      GROUP BY g
    ),
    ss_between AS (
      SELECT SUM(n * POWER(group_mean - grand_mean, 2)) AS ssb
      FROM by_group, overall
    ),
    ss_total AS (
      SELECT SUM(POWER(y - grand_mean, 2)) AS sst
      FROM base, overall
      WHERE y IS NOT NULL
    )
    SELECT
      CASE WHEN sst = 0 OR sst IS NULL THEN NULL ELSE ssb / sst END AS eta2
    FROM ss_between, ss_total
    """
    return con.execute(q).fetchone()[0]

rows = []
for c in candidates:
    eta2 = eta_squared_any(con, TABLE, TARGET, c)
    rows.append((c, eta2))

eta_df = (
    pd.DataFrame(rows, columns=["variable", "eta_squared"])
      .dropna(subset=["eta_squared"])
      .sort_values("eta_squared", ascending=False)
      .head(50)
      .reset_index(drop=True)
)

eta_df

#show only top 15 of the categorical variables
eta_df.head(15)


,variable,eta_squared
0,ESP,0.232762
1,PRIVCOV,0.221335
2,HINS4,0.206511
3,HINS1,0.192350
4,NOP,0.179474
5,SCHL,0.123159
6,PUBCOV,0.122245
7,MSP,0.075208
8,ESR,0.072523
9,HISP,0.071534


In [31]:
#reusable funtion to check how individual categorical variables and their specified categories are related to povpip
def summarize_by_cat(con, table, target, cat):
    q = f"""
    SELECT
      '{cat}' AS variable,
      CAST("{cat}" AS VARCHAR) AS group_value,
      COUNT(*) AS n,
      AVG(TRY_CAST("{target}" AS DOUBLE)) AS mean_target,
      MEDIAN(TRY_CAST("{target}" AS DOUBLE)) AS median_target
    FROM {table}
    WHERE "{target}" IS NOT NULL AND "{cat}" IS NOT NULL
    GROUP BY 1, 2
    ORDER BY n DESC
    """
    return con.execute(q).fetchdf()


In [32]:
#table summary of the ESP variable and each categories correlation to POVPIP
summarize_by_cat(con, TABLE, "POVPIP", top_var)


NameError: name 'top_var' is not defined

In [ ]:

df_age = con.execute("""
    SELECT
        AGEP,
        TRY_CAST(POVPIP AS DOUBLE) AS POVPIP
    FROM census_masterdb_all
    WHERE AGEP IS NOT NULL AND POVPIP IS NOT NULL
    ORDER BY random()
    LIMIT 50000
""").fetchdf()



In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

sns.scatterplot(
    data=df_age,
    x="AGEP",
    y="POVPIP",
    alpha=0.05
)
plt.title("AGE vs POVPIP")
plt.show()


In [ ]:
# frequency of values for each portion of the SCHL variable primarily based on years in school.

'''
# variable in the Census and its coded values
{
  "name": "SCHL",
  "label": "Educational attainment",
  "predicateType": "int",
  "group": "N/A",
  "limit": 0,
  "suggested-weight": "PWGTP",
  "values": {
    "item": {
      "04": "Grade 1",
      "03": "Kindergarten",
      "16": "Regular high school diploma",
      "01": "No schooling completed",
      "07": "Grade 4",
      "23": "Professional degree beyond a bachelor's degree",
      "22": "Master's degree",
      "19": "1 or more years of college credit, no degree",
      "10": "Grade 7",
      "02": "Nursery school, preschool",
      "0": "N/A (less than 3 years old)",
      "20": "Associate's degree",
      "21": "Bachelor's degree",
      "08": "Grade 5",
      "24": "Doctorate degree",
      "06": "Grade 3",
      "14": "Grade 11",
      "12": "Grade 9",
      "17": "GED or alternative credential",
      "09": "Grade 6",
      "11": "Grade 8",
      "13": "Grade 10",
      "18": "Some college, but less than 1 year",
      "15": "12th grade - no diploma",
      "05": "Grade 2"
    }
  }
}
'''

con.execute("""
    SELECT SCHL, COUNT(*) AS n
    FROM census_masterdb_all
    WHERE SCHL IS NOT NULL
    GROUP BY SCHL
    ORDER BY n DESC
    LIMIT 50
""").fetchdf()


In [ ]:
#assess educational attainment distributuions with POVPIP
df_schl = con.execute("""
SELECT
  SCHL,
  POVPIP
FROM census_masterdb_all
WHERE SCHL IS NOT NULL
  AND POVPIP IS NOT NULL
LIMIT 100000
""").fetchdf()


In [ ]:
#sanity check
df_schl.shape
df_schl.head()


In [ ]:
# force POVPIP to numeric for processing

df_schl["SCHL"] = pd.to_numeric(df_schl["SCHL"], errors="coerce")
df_schl["POVPIP"] = pd.to_numeric(df_schl["POVPIP"], errors="coerce")

df_schl.dtypes


In [ ]:
#drop nulls
df_schl = df_schl.dropna()


In [ ]:

df_schl = df_schl[(df_schl["SCHL"] >= 1) & (df_schl["SCHL"] <= 24)]


In [ ]:
#function to group numeric values into categoies based on the school (schl) variable
def schl_to_group(s):
    if 1 <= s <= 15:
        return "Less than HS"
    elif s in [16, 17]:
        return "HS or GED"
    elif s in [18, 19, 20]:
        return "Some college / Associate"
    elif s == 21:
        return "Bachelor"
    elif 22 <= s <= 24:
        return "Graduate"
    else:
        return None

df_schl["educ_group"] = df_schl["SCHL"].apply(schl_to_group)
df_schl = df_schl.dropna(subset=["educ_group"])


In [ ]:
#sanity check frequency of each category
df_schl["educ_group"].value_counts()


In [ ]:
import matplotlib.pyplot as plt

groups = [
    df_schl.loc[df_schl["educ_group"] == g, "POVPIP"]
    for g in ["Less than HS", "HS or GED", "Some college / Associate", "Bachelor", "Graduate"]
]

plt.figure(figsize=(8,5))
plt.boxplot(groups, labels=["<HS","HS","Some college","BA","Grad"], showfliers=False)
plt.yscale("log")
plt.ylabel("POVPIP")
plt.title("POVPIP by Educational Attainment")
plt.show()


In [ ]:
#force POVPIP to a numeric data type for the dataframe to do histogram comparison
df_pov["POVPIP"] = pd.to_numeric(df_pov["POVPIP"], errors="coerce")
df_pov["POVPIP"].dtype


In [ ]:
df_pov = con.execute("""
SELECT
  POVPIP
FROM census_masterdb_all
WHERE POVPIP IS NOT NULL
LIMIT 2000000
""").fetchdf()

df_pov["POVPIP"] = pd.to_numeric(df_pov["POVPIP"], errors="coerce")
df_pov = df_pov.dropna()


In [ ]:
plt.hist(df_pov["POVPIP"], bins=50)
plt.title("POVPIP distribution (first 50k rows)")
plt.show()
